# Scope of work

1) Import all necessary libraries and modules.  
2) First we need to obtain inforamtion about all available cars from the first main pages.  
3) Then we will call module dealers_cars to acquire links to every dealer's list of cars from all main pages.  
4) Now we will repeat the same procedure as in the 1st step but to the every dealer's cars list.  
5) Load all gathered information into a SQL database

## 1. Imort of libraries and modules

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from sqlalchemy import create_engine
import dealers_cars

## 2. Acquring cars' information from the first main pages

In [2]:
# Here we provide URL to the very first main page.

In [3]:
url = 'https://www.autoscout24.com/lst?atype=C&desc=0&sort=standard&source=homepage_search-mask&ustate=N%2CU'
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')

In [4]:
#get the number of total pages on the web site
def total_pages(soup):
    divs = soup.find('div', attrs={'class':'ListPage_pagination__v_4ci'})
    pages = divs.find_all('button', attrs={'class':'FilteredListPagination_button__41hHM'})[-2].text
    total_pages = int(pages)
    return total_pages

In [5]:
#get a dictionary of all URLs from every main page
all_pages = {}
for i in range(1, total_pages(soup) + 1):
    url_parts = url.split('&')
    url_parts[2] = f'page={i}'
    url = '&'.join(url_parts)
    all_pages[i]=url

In [6]:
#here we create a list of html codes as soup elements about all pages
soups_list = []
for k in all_pages:
    soups_list.append(BeautifulSoup(requests.get(all_pages[k]).text, 'lxml'))

In [7]:
# here we create blank lists to populate it later with cars' info
cars = []
characteristics = []
prices = []
locations = []

#this function scraps over the website in order to extract specific information about each car (characteristics, prices etc)
def parcing (tag, attr,df):
    for element in soups_list:
        info = element.find_all(tag, attrs={'class':attr})
        for i in info:
            df.append(i.get_text())
    return df

In [8]:
#here we call the above mentioned function and populate our previously created lists
cars = parcing('a', 'ListItem_title__znV2I ListItem_title_new_design__lYiAv Link_link__pjU1l', cars)
characteristics = parcing('div', 'VehicleDetailTable_container__mUUbY', characteristics)
prices = parcing('p', 'Price_price__WZayw PriceAndSeals_current_price__XscDn', prices)
locations = parcing('span', 'SellerInfo_address__txoNV', locations)

In [9]:
#here we extract only car's mark and maodel
for i in range(len(cars)):
    cars[i] = cars[i].split('\xa0')[0]

In [10]:
fuel_types = ['Gasoline','Diesel','Ethanol','Electric','Hydrogen','LPG','CNG','Electric/Gasoline','Others',
              'Electric/Diesel']
fuel_pattern = '|'.join(fuel_types)
gear = ['Automatic','Manual','Semi-automatic']
gear_pattern = '|'.join(gear)

In [11]:
#here we extract specific patterns of each car characteristics. The initial text that was extracted from web scraping
#contains too much unrelated data
for i in range(len(characteristics)):
    patterns = [r'\d{1,3}(?:,\d{3})*\s?km', f'({gear_pattern})', r'\d{1,2}/\d{4}', f'({fuel_pattern})', r'\d{1,4}\s?hp']
    characteristics[i] = [re.search(pattern, characteristics[i]).group(0).replace(',', '').replace(' km', '').replace(' hp', '').strip() if re.search(pattern, characteristics[i]) else None for pattern in patterns]

In [12]:
#here we extract integer from price text
for i in range(len(prices)):
    prices[i] = int(re.sub(r'\D', '', prices[i]))

In [13]:
#here we extract only country abbreveation
for i in range(len(locations)):
    try:
        locations[i] = locations[i].split('• ')[1].split('-')[0]
    except:
        locations[i] = locations[i].split('-')[0]

In [14]:
#Here we transform our lists into pandas Series
c = pd.Series(cars, name='Car')
ch = pd.Series(characteristics)
p = pd.Series(prices, name='Price [€]')
l = pd.Series(locations, name='Location')

In [15]:
# Create a DataFrame from the Series, which splits the lists into columns
df = pd.DataFrame(ch.tolist(), columns=['Mileage [km]', 'Transmission', 'Registration [m/y]', 'Fuel', 'Power [hp]'])

In [16]:
#Here we change data types of Mileage and Power to integers
try:
    df['Mileage [km]'] = df['Mileage [km]'].astype('int')
    df['Power [hp]'] = df['Power [hp]'].astype('int')
except:
    df['Power [hp]'] = df['Power [hp]'].fillna(0)
    df['Power [hp]'] = df['Power [hp]'].astype('int')
    df['Mileage [km]'] = df['Mileage [km]'].fillna(0)
    df['Mileage [km]'] = df['Mileage [km]'].astype('int')

In [17]:
merged_df = pd.concat([c, df], axis=1)
merged_df2 = pd.concat([merged_df,l], axis=1)
main_pages_info = pd.concat([merged_df2,p], axis=1)

In [18]:
main_pages_info

,Car,Mileage [km],Transmission,Registration [m/y],Fuel,Power [hp],Location,Price [€]
0,Porsche 911,2480,Automatic,07/2019,Gasoline,700,AT,569990
1,Audi Q7,260000,Automatic,10/2014,Diesel,245,BE,18950
2,Audi A3,99000,Manual,08/2018,Gasoline,150,BE,17500
3,Jaguar F-Pace,150964,Automatic,11/2016,Gasoline,340,BE,22990
4,Mercedes-Benz ML 250,130000,Automatic,10/2014,Diesel,204,BE,22950
...,...,...,...,...,...,...,...,...
395,Cupra Leon,20915,Automatic,07/2021,Electric,245,DE,26430
396,Mercedes-Benz E 53 AMG,68351,Automatic,03/2020,Gasoline,435,DE,67660
397,Skoda Fabia,57911,Manual,08/2018,Gasoline,95,BE,10990
398,BMW i4,24899,Automatic,12/2021,Electric,544,DE,79240


## Exporting parsed data to a SQL database

In [26]:
#configurations to connect to a SQL database
db_config = {
    'user': 'postgres', 
    'pwd': 'austria011020',
    'host': 'localhost',
    'port': 5432,
    'db': 'cars_small'
} 

In [27]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [28]:
engine = create_engine(connection_string)

In [29]:
with engine.connect() as conn:
    conn.execute('CREATE SCHEMA IF NOT EXISTS cars_small;')

In [30]:
with engine.connect() as conn:
    main_pages_info.to_sql(name='cars_info', con=conn, if_exists='replace', index=False)

## 2. Acquiring links to every dealer's list of cars from all main pages.

In [24]:
# We call the module dealers_cars and assign the URL from the first main page
dealers_cars_list = dealers_cars.sel_pars(url)

In [25]:
dealers_cars_list

['https://www.autoscout24.com/lst?atype=C&cid=5711',
 'https://www.autoscout24.com/lst?atype=C&cid=14789017',
 'https://www.autoscout24.com/lst?atype=C&cid=2364489',
 'https://www.autoscout24.com/lst?atype=C&cid=6934',
 'https://www.autoscout24.com/lst?atype=C&cid=15950',
 'https://www.autoscout24.com/lst?atype=C&cid=17172',
 'https://www.autoscout24.com/lst?atype=C&cid=20825',
 'https://www.autoscout24.com/lst?atype=C&cid=15250937',
 'https://www.autoscout24.com/lst?atype=C&cid=17700',
 'https://www.autoscout24.com/lst?atype=C&cid=4488',
 'https://www.autoscout24.com/lst?atype=C&cid=5525',
 'https://www.autoscout24.com/lst?atype=C&cid=5349679',
 'https://www.autoscout24.com/lst?atype=C&cid=5401905',
 'https://www.autoscout24.com/lst?atype=C&cid=4172834',
 'https://www.autoscout24.com/lst?atype=C&cid=34568',
 'https://www.autoscout24.com/lst?atype=C&cid=16391',
 'https://www.autoscout24.com/lst?atype=C&cid=15966',
 'https://www.autoscout24.com/lst?atype=C&cid=31057295',
 'https://www.a